Computing Similarities based on word frequency counts

In [132]:
from os import listdir
from gensim import corpora, models, similarities
import re 
import pickle

basepath = '/home/daniel/data/projects/datadive/data/texts/'
subfolderfiles = 'extracted/'
dictfn = 'termcounts-min-2-term-ids-map.txt'
recompute = True

matrixfn = 'matrix.pkl'
dictfn   = 'lookup.pkl'
corpusfn = 'corpus.pkl'
lsifn = 'lsi.pkl'
indexfn = 'index.pkl'
shards_prefix  = 'shards'


In [133]:
def transform_text(filename, word_dict=wdic):
    with open(filename, 'r') as f:
        file = []  # initiate a list
        for row in f.readlines():
            if len(row.split(',')) == 2:  # this get rid of rows with 3 values i.e. numbers
                row_val = row.split(',')
                row_val[1] = int(row_val[1].rstrip())
                row_val[0] = int(row_val[0])
                file.append(tuple(row_val))
    
    text = [i for i in file if i[0] in word_dict]
    return text

In [134]:
if recompute:
    corpus = pickle.load(open(basepath+corpusfn,'rb'))
    wdic = pickle.load(open(basepath+dictfn,'rb'))
        
    lsi = models.LsiModel(corpus, id2word=wdic, num_topics=256)
    index = similarities.MatrixSimilarity(lsi[corpus])
    
    with open(basepath+indexfn, 'wb') as f:
        pickle.dump(index, f)
    with open(basepath+corpusfn, 'wb') as f:
        pickle.dump(corpus, f)
    with open(basepath+lsifn, 'wb') as f:
        pickle.dump(lsi, f)
else:
    corpus = pickle.load(open(basepath+corpusfn,'rb'))
    lsi = pickle.load(open(basepath+lsifn,'rb'))
    index = pickle.load(open(basepath+indexfn,'rb'))
    

    


[(0, 1.0), (1, 0.84492612), (2, 0.41280794), (3, 0.87258351), (4, 0.92562878), (5, 0.92651176), (6, 0.90717238), (7, 0.93788087), (8, 0.91690272), (9, 0.84481013), (10, 0.88387662), (11, 0.95532817), (12, 0.91240776), (13, 0.83275837), (14, 0.87901217), (15, 0.88177866), (16, 0.85101265), (17, 0.92310977), (18, 0.92367488), (19, 0.90069646), (20, 0.92547858), (21, 0.88911474), (22, 0.91677272), (23, 0.044255227), (24, 0.011258808), (25, 0.020596206), (26, 0.034233965), (27, 0.019795917), (28, 0.028790861), (29, 0.028790861), (30, 0.021489549), (31, 0.043587252), (32, 0.046434708), (33, 0.028282709), (34, 0.044922832), (35, 0.029465947), (36, 0.030443016), (37, 0.032541264), (38, 0.036646042), (39, 0.040563118), (40, 0.070626453), (41, 0.070626453), (42, 0.062744096), (43, 0.016680801), (44, 0.0369104), (45, 0.022405017), (46, 0.039214309), (47, 0.049984645), (48, 0.037809044), (49, 0.044374023), (50, 0.072070956), (51, 0.037676632), (52, 0.036119856), (53, 0.025878463), (54, 0.02797100

In [135]:
sims = index[lsi[corpus[0]]]
print(list(enumerate(sims)))

[(0, 1.0), (1, 0.84492612), (2, 0.41280794), (3, 0.87258351), (4, 0.92562878), (5, 0.92651176), (6, 0.90717238), (7, 0.93788087), (8, 0.91690272), (9, 0.84481013), (10, 0.88387662), (11, 0.95532817), (12, 0.91240776), (13, 0.83275837), (14, 0.87901217), (15, 0.88177866), (16, 0.85101265), (17, 0.92310977), (18, 0.92367488), (19, 0.90069646), (20, 0.92547858), (21, 0.88911474), (22, 0.91677272), (23, 0.044255227), (24, 0.011258808), (25, 0.020596206), (26, 0.034233965), (27, 0.019795917), (28, 0.028790861), (29, 0.028790861), (30, 0.021489549), (31, 0.043587252), (32, 0.046434708), (33, 0.028282709), (34, 0.044922832), (35, 0.029465947), (36, 0.030443016), (37, 0.032541264), (38, 0.036646042), (39, 0.040563118), (40, 0.070626453), (41, 0.070626453), (42, 0.062744096), (43, 0.016680801), (44, 0.0369104), (45, 0.022405017), (46, 0.039214309), (47, 0.049984645), (48, 0.037809044), (49, 0.044374023), (50, 0.072070956), (51, 0.037676632), (52, 0.036119856), (53, 0.025878463), (54, 0.02797100